In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge
warnings.simplefilter(action='ignore', category=FutureWarning)
import gc

In [2]:
external2_train = pd.read_csv('../3_cleandata/train_df_elo17.csv')
external2_test = pd.read_csv('../3_cleandata/test_df_elo17.csv')

In [3]:
print(list(external2_train.columns))

['first_active_month', 'feature_1', 'feature_2', 'feature_3', 'target', 'outliers', 'quarter', 'elapsed_time', 'days_feature_1', 'days_feature_1_ratio', 'days_feature_2', 'days_feature_2_ratio', 'days_feature_3', 'days_feature_3_ratio', 'features_sum', 'features_mean', 'features_max', 'features_min', 'features_var', 'features_prod', 'new_subsector_id_nunique', 'new_merchant_id_nunique', 'new_merchant_category_id_nunique', 'new_month_mean', 'new_month_min', 'new_month_max', 'new_hour_nunique', 'new_hour_mean', 'new_hour_min', 'new_hour_max', 'new_weekofyear_nunique', 'new_weekofyear_mean', 'new_weekofyear_min', 'new_weekofyear_max', 'new_weekday_mean', 'new_weekday_min', 'new_weekday_max', 'new_day_nunique', 'new_day_mean', 'new_day_min', 'new_day_max', 'new_purchase_amount_sum', 'new_purchase_amount_max', 'new_purchase_amount_min', 'new_purchase_amount_mean', 'new_purchase_amount_var', 'new_purchase_amount_skew', 'new_installments_sum', 'new_installments_max', 'new_installments_mean', 

In [4]:
col2 = ['new_Christmas_Day_2017_mean', 'new_Christmas_Day_2017_sum', 'new_Children_day_2017_mean', 'new_Children_day_2017_sum', 'new_Black_Friday_2017_mean', 'new_Black_Friday_2017_sum', 'new_Mothers_Day_2018_mean', 'new_Mothers_Day_2018_sum','hist_Christmas_Day_2017_mean', 'hist_Christmas_Day_2017_sum', 'hist_Mothers_Day_2017_mean', 'hist_Mothers_Day_2017_sum', 'hist_fathers_day_2017_mean', 'hist_fathers_day_2017_sum', 'hist_Children_day_2017_mean', 'hist_Children_day_2017_sum', 'hist_Valentine_Day_2017_mean', 'hist_Valentine_Day_2017_sum', 'hist_Black_Friday_2017_mean', 'hist_Black_Friday_2017_sum', 'hist_Mothers_Day_2018_mean', 'hist_Mothers_Day_2018_sum']
external2_train = external2_train[col2]
external2_test = external2_test[col2]


In [5]:
internal_train = pd.read_csv('../3_cleandata/train_1.csv')
internal_test = pd.read_csv('../3_cleandata/test_1.csv')

In [6]:
print(list(internal_train.columns))

['feature_1', 'feature_2', 'feature_3', 'elapsed_time', 'hist_transactions_count', 'hist_category_1_sum', 'hist_category_1_mean', 'hist_category_2_1.0_mean', 'hist_category_2_2.0_mean', 'hist_category_2_3.0_mean', 'hist_category_2_4.0_mean', 'hist_category_2_5.0_mean', 'hist_category_3_A_mean', 'hist_category_3_B_mean', 'hist_category_3_C_mean', 'hist_merchant_id_nunique', 'hist_merchant_category_id_nunique', 'hist_state_id_nunique', 'hist_city_id_nunique', 'hist_subsector_id_nunique', 'hist_purchase_amount_sum', 'hist_purchase_amount_mean', 'hist_purchase_amount_max', 'hist_purchase_amount_min', 'hist_purchase_amount_std', 'hist_installments_sum', 'hist_installments_mean', 'hist_installments_max', 'hist_installments_min', 'hist_installments_std', 'hist_purchase_month_mean', 'hist_purchase_month_max', 'hist_purchase_month_min', 'hist_purchase_month_std', 'hist_purchase_date_ptp', 'hist_purchase_date_min', 'hist_purchase_date_max', 'hist_month_lag_min', 'hist_month_lag_max', 'auth_trans

In [7]:
internal_train.shape

(201917, 139)

In [8]:
external_test = pd.read_csv('../3_cleandata/test_ext1.csv')
external_train = pd.read_csv('../3_cleandata/train_ext1.csv')


In [9]:
target = pd.read_csv('../3_cleandata/target_int.csv',header= None)

In [10]:
external_train.shape

(201917, 215)

In [11]:
external_train.head()

,first_active_month,feature_1,feature_2,feature_3,target,outliers,quarter,elapsed_time,days_feature_1,days_feature_1_ratio,...,month_lag_min,category_1_mean,duration_mean,duration_min,duration_max,amount_month_ratio_mean,amount_month_ratio_min,amount_month_ratio_max,CLV_ratio,CLV_sq
0,2017-06-01,0.013145,0.00875,0.011430,-0.8203,0,2,604,3020,0.008280,...,-7.0,0.00000,-13.470,-16.78,5.540,-0.11050,-0.1330,0.04584,0.007045,108812.65000
1,2017-01-01,0.010710,0.01138,0.010284,0.3928,0,1,755,3020,0.005300,...,-11.0,0.08856,-16.160,-18.47,1.180,-0.11145,-0.1283,0.00815,0.000349,13576.58700
2,2016-08-01,0.010610,0.00875,0.010284,0.6880,0,3,908,1816,0.002203,...,-11.0,0.00000,-15.220,-16.44,-9.305,-0.12494,-0.1301,-0.07690,0.000562,7.21371
3,2017-09-01,0.010710,0.01417,0.010284,0.1425,0,3,512,2048,0.007812,...,-4.0,0.29880,-14.484,-16.92,2.562,-0.11960,-0.1384,0.02124,0.008454,1038.37830
4,2017-11-01,0.008060,0.01417,0.010284,-0.1598,0,4,451,451,0.002218,...,-2.0,0.16830,-12.310,-16.72,13.760,-0.10175,-0.1377,0.11380,0.071696,59269.74200


In [12]:
internal_train.head()

,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_category_1_sum,hist_category_1_mean,hist_category_2_1.0_mean,hist_category_2_2.0_mean,hist_category_2_3.0_mean,...,installments_min_mean,installments_min_std,installments_max_mean,installments_max_std,installments_std_mean,installments_std_std,authorized_flag_sum,authorized_flag_mean,card_id,first_active_month
0,1.597335,0.338839,0.876431,-0.466376,0.311084,-0.234615,-0.548342,1.203092,-0.209665,-0.432998,...,-0.098800,-0.041617,-0.085851,-0.053041,-0.024789,-0.035181,1.667028,0.494966,C_ID_92a2005557,2017-06-01
1,0.754276,-0.992082,-1.140992,0.047738,0.150070,0.011414,0.048638,0.798033,-0.209665,-0.432998,...,0.252809,0.279891,0.288626,0.225142,0.202897,0.323511,2.594044,0.657240,C_ID_3d0044924f,2017-01-01
2,-0.931842,0.338839,-1.140992,0.568661,-0.574497,-0.234615,-0.548342,-1.024733,-0.209665,-0.432998,...,-0.116381,-0.081152,-0.127460,-0.093761,NaN,NaN,-0.408683,0.525447,C_ID_d639edf6cd,2016-08-01
3,0.754276,1.669759,-1.140992,-0.779611,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.045938,0.931856,C_ID_186d6a6901,2017-09-01
4,-1.774901,1.669759,-1.140992,-0.987299,-0.332975,0.134429,1.421691,-1.024733,-0.209665,-0.432998,...,0.569258,0.672080,0.413451,0.484505,-0.065333,-0.085980,0.467952,0.603368,C_ID_cdbd2c0db2,2017-11-01


In [13]:
cols = ['hist_Christmas_Day_2017_mean','hist_Christmas_Day_2017_sum', 'hist_Mothers_Day_2017_mean',
 'hist_Mothers_Day_2017_sum', 'hist_fathers_day_2017_mean', 'hist_fathers_day_2017_sum',
 'hist_Children_day_2017_mean', 'hist_Children_day_2017_sum', 'hist_Valentine_Day_2017_mean',
 'hist_Valentine_Day_2017_sum', 'hist_Black_Friday_2017_mean', 'hist_Black_Friday_2017_sum',
 'hist_Mothers_Day_2018_mean', 'hist_Mothers_Day_2018_sum', 'CLV_ratio',
 'CLV_sq', 'new_Christmas_Day_2017_mean', 'new_Christmas_Day_2017_sum',
 'new_Children_day_2017_mean', 'new_Children_day_2017_sum', 'new_Black_Friday_2017_mean','new_Black_Friday_2017_sum',
 'new_Mothers_Day_2018_mean', 'new_Mothers_Day_2018_sum',]

def joining_dataset(internal, external,external2,cols):
    combine = pd.concat([external2,internal,external[cols]] ,axis=1, join='inner')
    return combine

train = joining_dataset(internal= internal_train, external= external_train,external2 = external2_train, cols = cols)
test = joining_dataset(internal= internal_test, external= external_test,external2 = external2_test, cols = cols)

In [14]:
train.shape

(201917, 185)

In [15]:
test.shape

(123623, 185)

In [16]:
 _, i = np.unique(train.columns, return_index=True)

train_df =train.iloc[:, i]
train_df['target'] = target
print(train_df.shape)
 _, j = np.unique(test.columns, return_index=True)
test_df =test.iloc[:, j]
print(test_df.shape)

/home/tung/miniconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


(201917, 164)
(123623, 163)


In [17]:
train_df.replace([np.inf, -np.inf], np.nan,inplace = True)
train_df.fillna(0,inplace = True)

/home/tung/miniconda3/envs/kaggle/lib/python3.7/site-packages/pandas/core/frame.py:4032: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/home/tung/miniconda3/envs/kaggle/lib/python3.7/site-packages/pandas/core/frame.py:4024: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [18]:
#exclude the card_id and first_active month in the features 
features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]
#create a list of categorical features 
categorical_feats = [c for c in features if 'feature_' in c]

In [19]:
train_df['outlier'] =train_df['target'].apply(lambda x: 1 if x<-30 else 0)

/home/tung/miniconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
target_bin = train_df['outlier']

In [21]:
outlier_card = train_df[train_df['outlier'] ==1][['card_id','outlier']]

In [22]:
del train_df['target']
del train_df['outlier']

In [23]:
# #simple train test split

# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(train_df, target_bin, test_size=0.33, random_state=42,stratify = target_bin)

In [24]:
# y_train.value_counts()

In [25]:
# from annoy import AnnoyIndex
# import random

# # for i in xrange(1000):
# #     v = [random.gauss(0, 1) for z in xrange(f)]
# #     t.add_item(i, v)

# mapper = {}
# i = 0
# for index, row in X_train.iterrows():
#     mapper[i] = row['card_id']
#     i +=1

# del X_train['card_id']
# X_train  = X_train.select_dtypes(include=['float'])
# f =  X_train.shape[1]
# t = AnnoyIndex(f)  # Length of item vector that will be indexed

# train_list = X_train.values.tolist()


# for k in range(len(train_list)):
#     t.add_item(k,train_list[k])
# t.build(10) # 10 trees
# # t.save('test.ann')

# # # ...

# # u = AnnoyIndex(f)
# # u.load('test.ann') # super fast, will just mmap the file
# # print(u.get_nns_by_item(0, 1000)) # will find the 1000 nearest neighbors

In [26]:
# print(t.get_nns_by_item(2, 10))

In [27]:
# outlier_card.head()

In [28]:
#get the list of outlier card
# outlier_list = outlier_card['card_id'].values.tolist()

In [29]:
# test_cardid = X_test['card_id']

# X_test  = X_test.select_dtypes(include=['float'])
# X_test['card_id'] = test_cardid #add card_id to be the last column
# #drop card_id like the one above 
# # del X_test['card_id']

In [30]:
# test_list = X_test.values.tolist()

In [31]:
# t.get_nns_by_vector(test_list[0],10, include_distances=False)

In [32]:
def return_card_id(nn_list,mapper,outlier_list):
    card_list = []
    score = 0
#     print('length of outlier list : {}'.format(len(outlier_list)))
    for nn in nn_list:
#         print(nn)
        card_list.append(mapper[nn])
    for card in card_list:
        if card in outlier_list:
            score +=1 
            
    return score

# # tung = return_card_id([24219, 22766, 74027, 88883, 108875, 59035, 126303, 37529, 127449, 116035],mapper,outlier_list)

In [33]:
# X_test.head()

In [34]:
# X_test['embedding'] = X_test.iloc[:,:-1].values.tolist()
# # 
# # X_test = X_test.iloc[ :, -2:]
# X_test.head()

In [35]:
# X_test['nn_neighbor'] =  X_test['embedding'].apply(lambda x : t.get_nns_by_vector(x,10, include_distances=False))


# t.get_nns_by_vector(test_list[0],10, include_distances=False)

In [36]:
# X_test.head()

In [37]:
# X_test['score'] = X_test['nn_neighbor'].apply(lambda x :return_card_id(x,mapper,outlier_list))

In [38]:
# X_test.head()

In [39]:
# X_test.iloc[:,:-1].head()

In [40]:
# X_test['true_label']= y_test

In [41]:
# X_test.head()

In [42]:
# X_test['acc'] = 0
# X_test.loc[(X_test['score'] != 0) & (X_test['true_label'] == 1),'acc'] = 1
    
# # X_test['acc'] = X_test.apply(lambda row : accuracy_model(row))
# # X_test.head()

In [43]:
# X_test['acc'].value_counts()

In [44]:
# X_test['true_label'].value_counts()

In [45]:
# X_test[(X_test['score'] !=0) & (X_test['true_label'] ==0)].head()

In [46]:
#calculate true positive 
def true_pos(threshold, score, true_label,df):
    true_pos = df[(df[score] >= threshold) & (df[true_label] ==1)].shape[0]
#     print(true_pos)
    return true_pos

def true_neg(threshold, score, true_label,df):
    tn = df[(df[score] < threshold) & (df[true_label] ==0)].shape[0]
#     print(tn)
    return tn


def false_pos(threshold, score, true_label,df):
    fp =df[(df[score] >=threshold) & (df[true_label] ==0)].shape[0]
#     print(fp)
    return fp

def false_neg(threshold, score, true_label,df):
    fn = df[(df[score] <threshold) & (df[true_label] ==1)].shape[0]
#     print(fn)
    return  fn

def cal_precision(fp,fn,tp,tn):
    
    precision = tp/(tp + fp)
    return precision

def cal_recall(fp,fn,tp,tn):
    recall = tp/(tp + fn)
    return recall
def f1_score(precision, recall):
    if (precision + recall) == 0:
        return 'error'
    else:
        f1 = 2*(precision*recall) /(precision + recall)
        return f1
# X_test[(X_test['score'] >=1) & (X_test['true_label'] ==0)]

In [47]:
# max_thres = X_test['score'].max()

# for thres in range(max_thres):
#     print ('for threshold = {} =========='.format(thres))
#     tp =  true_pos(thres,'score','true_label',X_test)
#     tn =  true_neg(thres,'score','true_label',X_test)
#     fp =  false_pos(thres,'score','true_label',X_test)
#     fn =  false_neg(thres,'score','true_label',X_test)
#     precision = cal_precision(fp,fn,tp,tn)
#     recall = cal_recall(fp,fn,tp,tn)
#     f1 = f1_score(precision, recall)
#     print((tp+tn)/(fp+ tp+ fn+ tn))
#     print(tp , tn, fp,fn,precision,recall,f1)

In [48]:
from sklearn.model_selection import StratifiedKFold
from annoy import AnnoyIndex

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_df,target_bin)):
    print('fold {} ======================================================================== '.format(n_fold))
    trn_x, trn_y = train_df.iloc[trn_idx], target_bin.iloc[trn_idx]
    val_x, val_y = train_df.iloc[val_idx], target_bin.iloc[val_idx]
    
    mapper = {}
    i = 0
    for index, row in trn_x.iterrows():
        mapper[i] = row['card_id']
        i +=1
    print('done with mapper.......')
    del trn_x['card_id']
    trn_x  = trn_x.select_dtypes(include=['float'])
    f =  trn_x.shape[1]
    t = AnnoyIndex(f)  # Length of item vector that will be indexed

    train_list = trn_x.values.tolist()


    for k in range(len(trn_x)):
        t.add_item(k,train_list[k])
    t.build(10)
    print('done with building tree.....')
    
    #get the list of outlier card
    outlier_list = outlier_card['card_id'].values.tolist()
    test_cardid = val_x['card_id']

    val_x  = val_x.select_dtypes(include=['float'])
    val_x['card_id'] = test_cardid #add card_id to be the last column
    
    val_x['embedding'] = val_x.iloc[:,:-1].values.tolist() #add embedding column
    val_x = val_x.iloc[ :, -2:]
    
    val_x['nn_neighbor'] =  val_x['embedding'].apply(lambda x : t.get_nns_by_vector(x,10, include_distances=False))

    val_x['score'] = val_x['nn_neighbor'].apply(lambda x :return_card_id(x,mapper,outlier_list))
    
    val_x['true_label']= val_y
    
    max_thres = val_x['score'].max()
    gc.collect()
    for thres in range(max_thres):
        print ('for threshold = {} =========='.format(thres))
        tp =  true_pos(thres,'score','true_label',val_x)
        tn =  true_neg(thres,'score','true_label',val_x)
        fp =  false_pos(thres,'score','true_label',val_x)
        fn =  false_neg(thres,'score','true_label',val_x)
        precision = cal_precision(fp,fn,tp,tn)
        recall = cal_recall(fp,fn,tp,tn)
        f1 = f1_score(precision, recall)
        print('true positive :{}'.format(tp))
        print('true negative :{}'.format(tn))
        print('false positive :{}'.format(fp))
        print('false negative :{}'.format(fn))
        print('accuracy : {}'.format((tp+tn)/(fp+ tp+ fn+ tn)))
        print('recall : {}'.format(recall))
        print('F1 score :{}'.format(f1))
#         print(tp , tn, fp,fn,precision,recall,f1)

fold 0 ======================================================================== 
done with mapper.......
done with building tree.....
for threshold = 0 ==========
true positive :442
true negative :0
false positive :39942
false negative :0
accuracy : 0.010944928684627575
recall : 1.0
F1 score :0.02165286827021996
for threshold = 1 ==========
true positive :121
true negative :36379
false positive :3563
false negative :321
accuracy : 0.903823296354992
recall : 0.2737556561085973
F1 score :0.058652447891420254
for threshold = 2 ==========
true positive :26
true negative :39502
false positive :440
false negative :416
accuracy : 0.9788034865293186
recall : 0.058823529411764705
F1 score :0.05726872246696035
for threshold = 3 ==========
true positive :2
true negative :39886
false positive :56
false negative :440
accuracy : 0.9877179080824089
recall : 0.004524886877828055
F1 score :0.008
for threshold = 4 ==========
true positive :0
true negative :39935
false positive :7
false negative :442
acc

In [49]:
df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), columns=['a', 'b', 'c'])
    
df2.head()

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [50]:
df2['d'] = df2.iloc[:,:-1].values.tolist()
df2 = df2.iloc[ :, -1:]
df2

,d
0,"[1, 2]"
1,"[4, 5]"
2,"[7, 8]"
